# Guia Prático de Pré-processamento de Texto (PT-BR) — Versão Simplificada

Este notebook é um guia didático para aprender **NLP aplicado à classificação de texto**.  
Etapas incluídas:

1. Exploração inicial dos dados (EDA)
2. Limpeza e normalização de texto
3. Tokenização, stopwords, stemming e lematização
4. Frequência de palavras
5. Representação com TF–IDF
6. Treino/teste com Naive Bayes (baseline)

---

### Conceitos-chave

- **Tokenização** → dividir o texto em palavras.  
- **Stopwords** → palavras muito comuns que não ajudam na classificação (ex.: "de", "a", "o").  
- **Stemming** → reduz a palavra para uma raiz (ex.: "agredindo" → "agred").  
- **Lematização** → reduz para a forma dicionário (ex.: "agredindo" → "agredir").  
- **n-grams** → combinações de n palavras consecutivas (1=unigram, 2=bigram).  
- **TF–IDF** → transforma texto em vetores.  
  - TF = frequência de um termo no documento  
  - IDF = quão raro é no corpus  
  - resultado: destaca termos que são **característicos** de cada classe.


## 0) Instalação (se necessário)

Se estiver no Google Colab, descomente e rode:


In [ ]:
# !pip install -U spacy nltk unidecode scikit-learn
# !python -m spacy download pt_core_news_sm
# import nltk
# nltk.download('stopwords')

## 1) Imports e Configuração

In [ ]:
import re, string
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import spacy
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from unidecode import unidecode

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# carregar modelo spaCy em português
nlp = spacy.load("pt_core_news_sm")

# stopwords e stemmer do NLTK
PT_STOPWORDS = set(stopwords.words("portuguese"))
STEMMER = RSLPStemmer()

plt.rcParams["figure.figsize"] = (7,4)
print("Ambiente pronto!")

## 2) Carregar Dados de Exemplo

In [ ]:
# Mini dataset fictício (substitua pelo seu CSV real)
data = {
    "text": [
        "A vítima relatou ameaça grave por parte do companheiro via telefone.",
        "Durante a discussão, ocorreu lesão corporal com empurrões e tapas.",
        "O autor enviou mensagens intimidatórias, dizendo que iria machucar a vítima.",
        "Foi registrado boletim por agressão física, resultando em hematomas no braço.",
        "Sem indícios claros de crime, apenas desentendimento verbal entre as partes."
    ],
    "label": ["AMEAÇA", "LESÃO CORPORAL", "AMEAÇA", "LESÃO CORPORAL", "OUTRO"]
}
df = pd.DataFrame(data)
df.head()

## 3) EDA (Exploratory Data Analysis)

In [ ]:
print("Info:")
display(df.info())
print("\nDistribuição de classes:")
display(df['label'].value_counts())

# tamanho dos textos
df['n_chars'] = df['text'].apply(len)
df['n_words'] = df['text'].apply(lambda x: len(x.split()))
display(df[['n_chars','n_words']].describe())

plt.hist(df['n_words'], bins=10)
plt.title("Distribuição de tamanho dos textos (n_words)")
plt.show()

## 4) Limpeza de Texto

In [ ]:
def clean_text(text):
    t = text.lower()
    t = re.sub(r"(https?://\S+|www\.\S+)", " ", t)  # urls
    t = re.sub(r"\b[\w.-]+@[\w.-]+\.\w{2,}\b", " ", t)  # emails
    t = re.sub(r"@\w+", " ", t)  # menções
    t = re.sub(r"\d+", " ", t)  # números
    t = t.translate(str.maketrans("", "", string.punctuation))  # pontuação
    t = unidecode(t)  # acentos
    t = re.sub(r"\s+", " ", t).strip()
    return t

df['clean'] = df['text'].apply(clean_text)
df[['text','clean']].head()

## 5) Tokenização, Stopwords, Stemming e Lematização

In [ ]:
def tokenize(text): return text.split()
def remove_stop(tokens): return [t for t in tokens if t not in PT_STOPWORDS]
def stem_tokens(tokens): return [STEMMER.stem(t) for t in tokens]
def lemmatize(text): return [tok.lemma_ for tok in nlp(text)]

df['tokens'] = df['clean'].apply(tokenize)
df['nostop'] = df['tokens'].apply(remove_stop)
df['stem'] = df['nostop'].apply(stem_tokens)
df['lemmas'] = df['clean'].apply(lemmatize)

df[['clean','nostop','stem','lemmas']].head()

## 6) Frequência de Palavras

In [ ]:
from itertools import chain

def top_freq(token_lists, k=10):
    counts = Counter(chain.from_iterable(token_lists))
    return pd.DataFrame(counts.most_common(k), columns=['token','freq'])

top_freq(df['nostop'], k=15)

## 7) Representação com TF–IDF

In [ ]:
X_text = df['clean'].tolist()
y = df['label'].tolist()

tfidf = TfidfVectorizer(stop_words=list(PT_STOPWORDS), ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(X_text)
print("Shape da matriz TF-IDF:", X_tfidf.shape)

# termos com maior IDF (mais raros)
idf_sorted = sorted(zip(tfidf.idf_, tfidf.get_feature_names_out()), reverse=True)[:10]
pd.DataFrame(idf_sorted, columns=['idf','termo'])

## 8) Baseline com Naive Bayes

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_text, y, test_size=0.2, random_state=42, stratify=y)

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=list(PT_STOPWORDS), ngram_range=(1,2))),
    ("clf", MultinomialNB())
])

pipe.fit(X_train, y_train)
preds = pipe.predict(X_val)

print(classification_report(y_val, preds))
cm = confusion_matrix(y_val, preds, labels=sorted(set(y)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(set(y)))
disp.plot(values_format='d')
plt.show()

## 9) Próximos Passos

- Testar Logistic Regression e SVM
- Balancear classes
- Usar embeddings (BERT, LegalBERT-pt)
- Documentar sempre as escolhas!